In [ ]:
import pandas as pd
import plotly.express as px

pd.options.plotting.backend = "plotly"

df = pd.read_csv('block-times-oct-27-2023.zip', index_col=0)

In [ ]:
min_time = df.min(axis=1)
max_time = df.max(axis=1)
diff = max_time - min_time
fig = px.line(diff)
fig.update_layout(
    xaxis_title='Block',
    yaxis_title='Timsetamp delta(s)',
    showlegend=False,
)

fig.show()


While there was as significant delta of 400ms in one of the initial blocks this wasn't repeated so is probably safe to
ignore for now. There appears to be a somewhat consistently larger delta between ~800K blocks and 1.2M blocks.
It may be be a bit misleading because this graph is so dense.

Lets see if we can zoom in on those 800K to 1.2M block range.

In [ ]:
slow_area = px.line(diff[800_000:1_200_000])
slow_area.update_layout(
    xaxis_title='Block',
    yaxis_title='Timestamp delta(s)',
    showlegend=False,
)

slow_area.show()

This still isn't clear enough so we'll zoom in further to that high 200s point.

In [ ]:
zoomed_slow_area = px.line(diff[979_200:979_400])
zoomed_slow_area.update_layout(
    xaxis_title='Block',
    yaxis_title='Timestamp delta(s)',
    showlegend=False,
)

zoomed_slow_area.show()


Zooming in to a range of 200 blocks gives us a better idea. It looks like there was an an issue where ~30 consecutive
blocks took a bit longer.
If we look at the timestamps per node perhaps we can better understand what's going on.

In [ ]:
node_times = px.line(df[979270:979300].astype(dtype="datetime64[s]"))

node_times.update_layout(
    xaxis_title='Block',
    yaxis_title='Time (UTC)',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=4,
        xanchor="center",
        x=0.5
    )
)

node_times.show()

Looking at that output it appears that for whatever reason the IdeasBeyondBorders node was being delayed noticeably more
than the other nodes.
Hint: you can mouse over the graph to see the exact values and node name.

Let's spot check one other hotspot.

In [ ]:
zoomed_slow_tail = px.line(diff[1_168_900:1_169_000])
zoomed_slow_tail.update_layout(
    xaxis_title='Block',
    yaxis_title='Timestamp delta(s)',
    showlegend=False,
)

zoomed_slow_tail.show()


In [ ]:
node_times_2 = px.line(df[1_168_940:1_168_950].astype(dtype="datetime64[s]"))

node_times_2.update_layout(
    xaxis_title='Block',
    yaxis_title='Time (UTC)',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=4,
        xanchor="center",
        x=0.5
    )
)

node_times_2.show()